In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# plots show in the notebook
%matplotlib inline

# additional configuration (optional)
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'

In [2]:
ds_cm4_fwf = xr.open_dataset('/archive/Rebecca.Beadling/xanadu_om4_v1.0.5/CM4_piControl_C_fafantwater/gfdl.ncrc4-intel18-prod-openmp/history/00010101.ocean_month.nc')
ds_cm4_static = xr.open_dataset('/archive/Rebecca.Beadling/xanadu_om4_v1.0.5/CM4_piControl_C_fafantwater/gfdl.ncrc4-intel18-prod-openmp/history/00010101.ocean_static.nc')

ds_esm4_fwf = xr.open_dataset('/archive/Rebecca.Beadling/FMS2019.01.03_mom6_ESM4_v1.0.3_debug/ESM4_piControl_D_fafantwater/gfdl.ncrc4-intel18-prod-openmp/1x1m0d_1152x4a_1057x1o3/history/00010101.ocean_month.nc')
ds_esm4_static = xr.open_dataset('/archive/Rebecca.Beadling/FMS2019.01.03_mom6_ESM4_v1.0.3_debug/ESM4_piControl_D_fafantwater/gfdl.ncrc4-intel18-prod-openmp/1x1m0d_1152x4a_1057x1o3/history/00010101.ocean_static.nc')


/nbhome/Rebecca.Beadling/miniconda/envs/python38_latestxarray/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)


In [ ]:
prcme_adj_cm4 = ds_cm4_fwf['prcme_adj']
area_cm4 = ds_cm4_static['areacello']

prcme_adj_esm4 = ds_esm4_fwf['prcme_adj']
area_esm4 = ds_esm4_static['areacello']
prcme_adj_cm4, prcme_adj_esm4

In [ ]:
prcme_adj_cm4_sv = (prcme_adj_cm4*area_cm4)/1000/1000000
prcme_adj_cm4_sv.isel(time=0).plot()
total_fwf_cm4 = prcme_adj_cm4_sv.sum(dim='xh').sum(dim='yh')
total_fwf_cm4

In [ ]:
prcme_adj_esm4_sv = (prcme_adj_esm4*area_esm4)/1000/1000000
prcme_adj_esm4.isel(time=0).plot()
#prcme_adj_esm4_sv.isel(time=0).plot()
total_fwf_esm4 = prcme_adj_esm4_sv.sum(dim='xh').sum(dim='yh')
total_fwf_esm4

In [ ]:
scale_factor = total_fwf_cm4.isel(time=0)/total_fwf_esm4
scale_factor

In [ ]:
esm4_scaled_sv = prcme_adj_esm4_sv*scale_factor
esm4_scaled_sv.plot()
esm4_scaled_total = esm4_scaled_sv.sum(dim='xh').sum(dim='yh')
esm4_scaled_total

In [ ]:
esm4_scaled_kg_m2_s = (esm4_scaled_sv/area_esm4)*1000*1000000
esm4_scaled_kg_m2_s.plot()

In [ ]:
prcme_adj = esm4_scaled_kg_m2_s
prcme_adj.plot()

In [3]:
# transform prcme_adj data-array into dataset
prcme_adj = prcme_adj.to_dataset(name='prcme_adj',promote_attrs=True)
prcme_adj = prcme_adj.rename({'xh':'longitude', 'yh':'latitude'})

#expand time dimension from l=1 to l=12
prcme_adj = numpy.tile(prcme_adj[None,:,:],(12,1,1))
prcme_adj

NameError: name 'prcme_adj' is not defined

In [11]:
prcme_adj.to_netcdf("test4.nc",unlimited_dims='time')